In [ ]:
import numpy as np                           #Cálculos númericos
import matplotlib.pyplot as plt              #Creación de gráficos
from scipy import signal
import matplotlib.cm as cm                   #Mapas de color
import scipy.integrate                       #Realiza integraciones

# Definir las funciones
def func_a(x, delta=0.05):

    #Definición de los ejes de tiempos de las funciones a trozos
    t = np.arange(-1,6+delta,delta)
    t_1 = np.arange(-1, 0, delta)
    t_2 = np.arange(0, 3, delta)
    t_3 = np.arange(3, 5, delta)
    t_4 = np.arange(5, 6 + delta, delta)

    #Concatenación de las funciones a trozos
    f_t = np.concatenate((np.zeros(len(t_1)), 2 * np.ones(len(t_2)), -2 * np.ones(len(t_3)), np.zeros(len(t_4))))

    return (t,f_t)

def func_b(x, delta=0.05):

    t = np.arange(-1,1+delta,delta)
    t_1 = [-1]
    t_2 = np.arange(-1+delta,1,delta)
    t_3 = [1]
    f_t = np.concatenate((np.zeros(len(t_1)),-1*(t_2),np.zeros(len(t_3))))

    return (t,f_t)

def func_c(x, delta=0.05):

    t = np.arange(-2,6+delta,delta)
    t_1 = np.arange(-2,-1,delta)
    t_2 = np.arange(-1,1,delta)
    t_3 = np.arange(1,3,delta)
    t_4 = np.arange(3,5,delta)
    t_5 = np.arange(5,6+delta,delta)

    f_t = np.concatenate((np.zeros(len(t_1)),2*np.ones(len(t_2)),-2*(t_3)+4,-2*np.ones(len(t_4)),np.zeros(len(t_5))))

    return (t,f_t)

def func_d(x, delta=0.05):

    t = np.arange(-3,3+delta,delta)
    t_1 = np.arange(-3,-3+delta,delta)
    t_2 = np.arange(-3+delta,0,delta)
    t_3 = np.arange(0,3,delta)
    t_4 = np.arange(3,3+delta,delta)

    f_t = np.concatenate((np.zeros(len(t_1)),np.exp(t_2),np.exp(-t_3),np.zeros(len(t_4))))

    return (t,f_t)

# Función para convolucionar
def convolucionar(func1, func2):

    delta=0.05
    # Recuperamos t y f_t de cada función
    ta, x_t = func1(None,delta)
    tb, h_t = func2(None, delta)

    #Graficamos las funciones originales
    plt.figure()
    plt.subplot(1,2,1)
    plt.plot(ta,x_t)
    plt.subplot(1,2,2)
    plt.plot(tb,h_t)

    # Valores de entrada
    x = np.linspace(-10, 10, 500)
    #Se llaman a las funciones para calcular los valores de las mismas en los puntos de x
    f1 = func1(x)
    f2 = func2(x)

    #Rango de tiempo en el que se va a realizar la convolución entre las dos funciones func1 y func2.
    ini = min(ta)-(round(max(tb),2)-min(tb))
    fin = round(max(ta),2)+(round(max(tb),2)-min(tb))   #round redondea el valor máximo a 2 decimales
    print(ini)
    print(fin)

    #Crea el eje de tiempo para la convolución (puntos de evaluación de la convolución)
    eje_tau = np.arange(ini,fin+delta,delta)
    #Se crea un array vacío para almacenar los resultados de la convolución
    eje_conv = np.zeros(len(eje_tau))

    x_tau = x_t

    #Se refleja h_t
    h_mtau = np.flip(h_t)

    #Se gráfica x_t y h_t reflejada
    plt.figure()
    plt.subplot(1,2,1)
    plt.plot(ta,x_tau)
    plt.subplot(1,2,2)
    plt.plot(tb,h_mtau)

    #Se prepara la señal x_tau para la operación de convolución: Se añaden ceros a la señal x_tau para
    #que h_mtau se desplace a la izquierda y a la derecha sin perder información
    t_x_tau = np.concatenate((np.zeros(len(h_mtau)-1),x_tau,np.zeros(len(h_mtau)-1)))

    #Bucle que controla el desplazamiento (tau) de la señal h_mtau a lo largo de x_tau.
    for tau in range(0,len(x_tau)-1+len(h_mtau)-1):
      #Versión desplazada de h_mtau
      #Crea ceros adicionales para completar la señal a la misma longitud que t_x_tau.
      t_h_tmtau = np.concatenate((np.zeros(tau),h_mtau,np.zeros(len(x_tau)-1+len(h_mtau)-1-tau)))

      #Indice actual en eje_conv donde se almacenará el resultado de la convolución
      idx = tau+len(h_mtau)-1
      #Multiplicación punto a punto entre t_x_tau (señal original con ceros añadidos) y
      #t_h_tmtau (versión desplazada de h_mtau).
      prod = t_x_tau * t_h_tmtau
      #Se integra el producto prod a lo largo de eje_tau
      eje_conv[idx] = scipy.integrate.simps(prod, eje_tau)

      plt.figure()
      plt.subplot(2,1,1)
      plt.plot(eje_tau,t_x_tau,'g')               #Dibuja la señal t_x_tau en verde
      plt.plot(eje_tau,t_h_tmtau,'m')             #Dibuja la señal desplazada t_h_tmtau en magenta
      plt.plot(eje_tau,prod)                      #producto punto a punto de las señales
      plt.fill_between(eje_tau, prod,cmap=cm.bwr) #Rellena el área bajo el producto para ayudar a visualizar la multiplicación
      plt.plot(eje_tau[idx],0,'ro')               #Marca el punto actual de idx en rojo
      plt.subplot(2,1,2)
      plt.plot(eje_tau,eje_conv)                  #Dibuja el resultado acumulativo de la convolución
      plt.plot(eje_tau[idx],eje_conv[idx],'ro')   #Valor actual de la convolución en rojo
      plt.show()


# Menú de selección
def menu():
    print("Selecciona dos funciones para convolucionar:")
    print("1. Función a")
    print("2. Función b")
    print("3. Función c")
    print("4. Función d")

    eleccion1 = int(input("Elige la primera función (1-4): "))
    eleccion2 = int(input("Elige la segunda función (1-4): "))

    funciones = {1: func_a, 2: func_b, 3: func_c, 4: func_d}

    func1 = funciones.get(eleccion1, None)
    func2 = funciones.get(eleccion2, None)

    if func1 and func2:
        convolucionar(func1, func2)
    else:
        print("Selección inválida. Inténtalo de nuevo.")

menu()